**Code for resize the Sipakmed images -> 256 x 256**

In [1]:
#Libraries
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
#Funtion to adjust images size
def resize_images(img):
  dimen = img.shape[0],img.shape[1]

  if dimen[0] >= 500 or dimen[1] >= 500:#todo arriba de 500
    new_img = cv2.resize(img,(0,0),fx=0.45,fy = 0.45,interpolation = cv2.INTER_LINEAR)
    #red_img = cv2.resize(new_img,(0,0),fx=(1./0.45),fy = (1/0.45),interpolation = cv2.INTER_LINEAR)
  elif dimen[0] >= 400 or dimen[1] >= 400:#entre 400 - 499
    new_img = cv2.resize(img,(0,0),fx=0.5,fy = 0.5,interpolation = cv2.INTER_LINEAR)
    #red_img = cv2.resize(new_img,(0,0),fx=(1/0.5),fy = (1/0.5),interpolation = cv2.INTER_LINEAR)
  elif dimen[0] >= 300 or dimen[1] >= 300:#entre 300-399
    new_img = cv2.resize(img,(0,0),fx=0.6,fy = 0.6,interpolation = cv2.INTER_LINEAR)
    #red_img = cv2.resize(new_img,(0,0),fx=(1/0.6),fy = (1/0.6),interpolation = cv2.INTER_LINEAR)
  elif dimen[0] >= 200 or dimen[1] >= 200:
    new_img = cv2.resize(img,(0,0),fx=0.8,fy = 0.8,interpolation = cv2.INTER_LINEAR)
    #red_img = cv2.resize(new_img,(0,0),fx=(1/0.8),fy = (1/0.8),interpolation = cv2.INTER_LINEAR)
  elif dimen[0] >= 100 or dimen[1] >= 100:
    new_img = cv2.resize(img,(0,0),fx=1.2,fy = 1.2,interpolation = cv2.INTER_LINEAR)
    #red_img = cv2.resize(new_img,(0,0),fx=(1/1.2),fy = (1/1.2),interpolation = cv2.INTER_LINEAR)
  else:
    new_img = cv2.resize(img,(0,0),fx=2,fy = 2,interpolation = cv2.INTER_LINEAR)
    #red_img = cv2.resize(new_img,(0,0),fx=(1/2),fy = (1/2),interpolation = cv2.INTER_LINEAR)

  return new_img

In [23]:
#Zero padding function
def zero_padding(new_img):#imagenes redimensinadas
  filas = (256-(new_img.shape[0]))//2#filas
  colum = (256-(new_img.shape[1]))//2#columnas
  padd = 255#valor del padding, puede ser 0,255

  try:
    img_rell1 = np.pad(new_img[:,:,0],((filas, filas),(colum,colum)),'constant', constant_values=(padd,padd))#capas
    img_rell2 = np.pad(new_img[:,:,1],((filas, filas),(colum,colum)),'constant', constant_values=(padd,padd))
    img_rell3 = np.pad(new_img[:,:,2],((filas, filas),(colum,colum)),'constant', constant_values=(padd,padd))
    img_rell = np.stack((img_rell1,img_rell2,img_rell3),axis=2)#juntar las capas de las imagenes
  except:
    img_rell = np.pad(new_img,((filas, filas),(colum,colum)),'constant', constant_values=(padd,padd))#mascaras

  return img_rell

In [24]:
#Add or eliminate zeros in rows
def adjustment_filas(img_rell):
  filas_aux = img_rell.shape[0]
  columnas_aux = img_rell.shape[1]
  bandera = 0

  try:
    img_rell1 = img_rell[:,:,0]
    img_rell2 = img_rell[:,:,1]
    img_rell3 = img_rell[:,:,2]

    #print(filas_aux,columnas_aux)
    if filas_aux < 256:#255 < 256 si es mas pequeño que 256
      #aux_zeros = np.zeros(columnas_aux,dtype=int)
      aux_zeros = np.ones(columnas_aux,dtype=int)
      img_rell1_n =  np.insert(img_rell1,-1,aux_zeros,axis=0)
      img_rell2_n =  np.insert(img_rell2,-1,aux_zeros,axis=0)
      img_rell3_n =  np.insert(img_rell3,-1,aux_zeros,axis=0)
    else:
      img_rell1_n = img_rell1
      img_rell2_n = img_rell2
      img_rell3_n = img_rell3
      bandera = 1

    img_rell_1 = np.stack((img_rell1_n,img_rell2_n,img_rell3_n),axis=2)#lo que regresa
  except:
    if filas_aux < 256:#255 < 256 si es mas pequeño que 256
      #aux_zeros = np.zeros(columnas_aux,dtype=int)
      #aux_zeros = np.ones(columnas_aux,dtype=int)
      aux_zeros = np.full((columnas_aux),255,dtype=int)
      img_rell_n = np.insert(img_rell,-1,aux_zeros,axis=0)
    else:
      img_rell_n = img_rell
      bandera = 1

    img_rell_1 = img_rell_n#lo que regresa

  return img_rell_1,bandera

In [25]:
#Add or eliminate zeros in columns
def adjustment_columnas(img_rell_1):
  filas_aux = img_rell_1.shape[0]
  columnas_aux = img_rell_1.shape[1]
  bandera = 0

  try:
    img_rell1 = img_rell_1[:,:,0]
    img_rell2 = img_rell_1[:,:,1]
    img_rell3 = img_rell_1[:,:,2]

    #print(filas_aux,columnas_aux)
    if columnas_aux < 256:#255 < 256 si es mas pequeño que 256
      #aux_zeros = np.zeros(filas_aux,dtype=int)
      aux_zeros = np.ones(filas_aux,dtype=int)
      aux_zeros = np.transpose(aux_zeros)
      img_rell1_n =  np.insert(img_rell1,-1,aux_zeros,axis=1)
      img_rell2_n =  np.insert(img_rell2,-1,aux_zeros,axis=1)
      img_rell3_n =  np.insert(img_rell3,-1,aux_zeros,axis=1)
    else:
      img_rell1_n = img_rell1
      img_rell2_n = img_rell2
      img_rell3_n = img_rell3
      bandera = 1

    img_rell_n = np.stack((img_rell1_n,img_rell2_n,img_rell3_n),axis=2)
  except:
    if columnas_aux < 256:#255 < 256 si es mas pequeño que 256
      #aux_zeros = np.zeros(filas_aux,dtype=int)
      #aux_zeros = np.ones(filas_aux,dtype=int)
      aux_zeros = np.full((filas_aux),255,dtype=int)
      aux_zeros = np.transpose(aux_zeros)
      img_rell_n = np.insert(img_rell_1,-1,aux_zeros,axis=1)
    else:
      img_rell_n = img_rell_1
      bandera = 1

    #img_rell_n = img_rell_n#lo que regresa

  return img_rell_n,bandera

In [ ]:
#Folds
index = 0   #cambiar de carpeta

carpetas = os.listdir('/content/drive/MyDrive/SipakMed_data')
carpetas.sort()
path = '/content/drive/MyDrive/SipakMed_data/'+carpetas[index]+'/CROPPED/'
dir_Masc = path+'Mascaras/maskCito'#carpeta de las mascaras
dir_Masn = path+'Mascaras/maskNuc'
dir_Masf = path+'Mascaras/maskFon'

dir_Rezc = path+'Resize/maskCito'#carpeta de imagenes_resize
dir_Rezn = path+'Resize/maskNuc'
dir_Rezf = path+'Resize/maskFon'
dir_Rezimg = path+'Resize/imagenes'

elemen = os.listdir(path)#elementos de directorio de imagenes
elemen.sort()#ordenar elementos de directorio
ele_Mcito = os.listdir(dir_Masc)#elementos de directorio de mascaras
ele_Mcito.sort()#ordenar elementos de directorio de mascaras
ele_Mnuc = os.listdir(dir_Masn)
ele_Mnuc.sort()
ele_Mfon = os.listdir(dir_Masf)
ele_Mfon.sort()

print(carpetas)
print(elemen)
print(len(elemen))
print(len(ele_Mnuc))
print(len(ele_Mfon))
print(dir_Masc)
print(dir_Rezc)
print(ele_Mcito)

['im_Dyskeratotic', 'im_Koilocytotic', 'im_Metaplastic', 'im_Parabasal', 'im_Superficial-Intermediate']
['001_01.bmp', '001_01_cyt.dat', '001_01_nuc.dat', '001_02.bmp', '001_02_cyt.dat', '001_02_nuc.dat', '001_03.bmp', '001_03_cyt.dat', '001_03_nuc.dat', '001_04.bmp', '001_04_cyt.dat', '001_04_nuc.dat', '001_05.bmp', '001_05_cyt.dat', '001_05_nuc.dat', '002_01.bmp', '002_01_cyt.dat', '002_01_nuc.dat', '002_02.bmp', '002_02_cyt.dat', '002_02_nuc.dat', '002_03.bmp', '002_03_cyt.dat', '002_03_nuc.dat', '002_04.bmp', '002_04_cyt.dat', '002_04_nuc.dat', '002_05.bmp', '002_05_cyt.dat', '002_05_nuc.dat', '002_06.bmp', '002_06_cyt.dat', '002_06_nuc.dat', '002_07.bmp', '002_07_cyt.dat', '002_07_nuc.dat', '002_08.bmp', '002_08_cyt.dat', '002_08_nuc.dat', '002_09.bmp', '002_09_cyt.dat', '002_09_nuc.dat', '002_10.bmp', '002_10_cyt.dat', '002_10_nuc.dat', '003_01.bmp', '003_01_cyt.dat', '003_01_nuc.dat', '003_02.bmp', '003_02_cyt.dat', '003_02_nuc.dat', '003_03.bmp', '003_03_cyt.dat', '003_03_nuc.d

In [ ]:
#Change size

name_img = [x for x in elemen if x.endswith('.bmp')] # Use a list comprehension to filter out elements

print(name_img)
print(len(name_img))

#for i in range(1,len(name_img)+1):
for i in range(1,5):
  print(i)
  #Lectura de info
  img = cv2.imread(path+name_img[i-1])

  #Funciones
  new_img = resize_images(img)
  img_rell = zero_padding(new_img)
  img_rell_1, bandera = adjustment_filas(img_rell)
  img_rell_n, bandera = adjustment_columnas(img_rell_1)

  #Nombres de nuevas imagenes (mascaras)
  file_imgRe_name = name_img[i-1].split('.')[0]+'_rsze.bmp'

  #Guardar
  os.chdir(dir_Rezimg)
  cv2.imwrite(file_imgRe_name,img_rell_n)#guardar imagen nueva con ceros
  #print(img_rell.shape)
  #print(img_rell_n.shape)
  if img_rell_n.shape != (256,256,3):
    print('Error')


